## 从视频帧中提取文字

In [4]:
import cv2
import pytesseract
import os

# 设置 Tesseract OCR 的路径（如果需要）
# pytesseract.pytesseract.tesseract_cmd = r'/usr/local/bin/tesseract'  # 根据实际安装路径设置

# 提取视频帧中的文字
def extract_text_from_frames(frame_folder, output_path):
    all_text = []
    for frame_file in sorted(os.listdir(frame_folder)):
        if frame_file.endswith('.jpg'):
            img_path = os.path.join(frame_folder, frame_file)
            img = cv2.imread(img_path)
            text = pytesseract.image_to_string(img)
            all_text.append(text)
    with open(output_path, 'w', encoding='utf-8') as f:
        f.write('\n'.join(all_text))
    print(f"Extracted text saved to {output_path}")

# 示例：提取某一视频帧文件夹中的文字
frame_folder = '../Datas/extract_hate_frame'
output_text_path = '../Datas/text_output/example_video_frames_text_hate.txt'
extract_text_from_frames(frame_folder, output_text_path)

Extracted text saved to ../Datas/text_output/example_video_frames_text_hate.txt


In [7]:
# import speech_recognition as sr
# from pydub import AudioSegment
# import os

# # 转换音频格式（如果需要）
# def convert_audio_format(input_path, output_path, target_format='wav'):
#     audio = AudioSegment.from_file(input_path)
#     audio.export(output_path, format=target_format)
#     print(f"Converted audio saved to {output_path}")

# # 提取音频中的文字
# def extract_text_from_audio(audio_path, output_path):
#     recognizer = sr.Recognizer()
#     audio_file = sr.AudioFile(audio_path)
    
#     with audio_file as source:
#         audio_data = recognizer.record(source)
    
#     try:
#         text = recognizer.recognize_google(audio_data, language='zh-CN')  # 假设语音为中文
#         with open(output_path, 'w', encoding='utf-8') as f:
#             f.write(text)
#         print(f"Extracted text saved to {output_path}")
#     except sr.UnknownValueError:
#         print("Google Speech Recognition could not understand the audio")
#     except sr.RequestError as e:
#         print(f"Could not request results from Google Speech Recognition service; {e}")

# # 示例：提取某一音频文件中的文字
# audio_input_path = '../Datas/extract_audio_files/hate_video_2.mp3'
# audio_output_wav_path = '../Datas/audio_output/hate_video_2.wav'
# text_output_path = '../Datas/text_output/hate_video_2.txt'

# # 转换音频格式为 wav（如果需要）
# convert_audio_format(audio_input_path, audio_output_wav_path)

# # 提取音频中的文字
# extract_text_from_audio(audio_output_wav_path, text_output_path)


Converted audio saved to ../Datas/audio_output/hate_video_2.wav
Google Speech Recognition could not understand the audio


In [16]:
# 安装 Whisper
!pip install git+https://github.com/openai/whisper.git
# 安装 Torch
!pip install torch
# 安装 Torchaudio
!pip install torchaudio

  Cloning https://github.com/openai/whisper.git to /private/var/folders/t2/6ybycps95j5g5q_zmwwpmtjh0000gn/T/pip-req-build-oam768yp
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /private/var/folders/t2/6ybycps95j5g5q_zmwwpmtjh0000gn/T/pip-req-build-oam768yp
  Resolved https://github.com/openai/whisper.git to commit ba3f3cd54b0e5b8ce1ab3de13e32122d0d5f98ab
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 907.0/907.0 kB 12.3 MB/s eta 0:00:00a 0:00:01
  Created wheel for openai-whisper: filename=openai_whisper-20231117-py3-none-any.whl size=802826 sha256=a87ff051fc583b8c0682cf61bd9037dcbdc76c3f9ab39c75f059ceac91e4d3b6
  Stored in directory: /private/var/folders/t2/6ybycps95j5g5q_zmwwpmtjh0000gn/T/pip-ephem-wheel-cache-dxm45zn9/wheels/1f/1d/98/9583695e6695a6ac0ad42d87511097dce5ba486647dbfecb0e
Successfully built open

In [19]:
import os
import whisper
from pydub import AudioSegment

# 设置文件夹路径
audio_folder_path = '../Datas/extract_audio_files'
output_folder_path = '../Datas/text_output'

# 创建输出文件夹（如果不存在）
os.makedirs(output_folder_path, exist_ok=True)

# 加载 Whisper 模型
model = whisper.load_model("base")

# 遍历文件夹中的所有音频文件
for audio_file in os.listdir(audio_folder_path):
    if audio_file.endswith('.mp3'):
        # 设置文件路径
        audio_input_path = os.path.join(audio_folder_path, audio_file)
        audio_output_wav_path = os.path.join(audio_folder_path, audio_file.replace('.mp3', '.wav'))
        text_output_path = os.path.join(output_folder_path, audio_file.replace('.mp3', '_text.txt'))
        
        # 转换音频格式
        audio = AudioSegment.from_file(audio_input_path)
        audio = audio.set_frame_rate(16000).set_channels(1)
        audio.export(audio_output_wav_path, format='wav')
        print(f"Converted audio saved to {audio_output_wav_path}")

        # 加载并转录音频文件
        result = model.transcribe(audio_output_wav_path)

        # 打印转录文本
        print(f"Transcribed text for {audio_file}:", result['text'])

        # 保存转录文本到文件
        with open(text_output_path, 'w', encoding='utf-8') as f:
            f.write(result['text'])
        print(f"Transcribed text saved to {text_output_path}")


Converted audio saved to ../Datas/extract_audio_files/non_hate_video_55.wav


/Users/cassie/anaconda3/lib/python3.11/site-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Transcribed text for non_hate_video_55.mp3:  Yeah, lack of challenge baby, huh? Trump, put up, put up, Trump, huh? I'm tuning in to CNN to see what they say next They call me homophobic, but I love gay sex They call me racist, but my wife's son's black Drag queens, show me love, I show them love back And I don't get it, they call me anti-Semitic, but I love Jews Honestly, no one's above Jews, Israel's the greatest ally Our country could ever have, and I gladly die for Put me in a body bag, give them all our money You're 11 million a day, man, that's not enough We need a triple of the pay And the last thing I'm gonna say, before I go away Is I don't mind being replaced as long as it's legal aid Hey, that's the answer, that bag of boy, Hey, tell me what you want, cause I don't matter, boy Hey, I just feel the truth that something down, I understand All this white livers trying to tell me who I am Come with me in groups, cause I'm on my own, man
Transcribed text saved to ../Datas/text_out

/Users/cassie/anaconda3/lib/python3.11/site-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Transcribed text for non_hate_video_41.mp3:  There's more trouble at a Brighton school which attracted worldwide attention after the Eggboy attack. Seven users learnt the headmaster there is now under fire for using a racist slur at a school assembly. Words are a matter of principle for this Brighton College educator. We're looking for not just what has been communicated but how has been communicated. But his words have landed him in hot water using a racist slur not once, but three times at school assembly. Richard Minak used the N word in the wake of the Christchurch terror attack to highlight insidious racism. So mission bread is only a tiny step in language or way from 1400 students were there including the boy who egged Senator Fraser many gasped in shock. You can't say that in front of a school especially a school as multicultural as ours. Mr Minak later emailed students to apologise and explain. I hope you understand that I used it to call out and criticize racism and bigotry. S

...

Transcribed text for hate_video_194.mp3:  I see them feeding fake news to the people I love. And I'm like, fuck, Jews. Who knew who? I guess the six fucking Gorylian wasn't enough. I'm like, fuck, Jews, and I, fuck niggers too. Said I'm the next aid I'll pitlie. I'm coming to get ya. Oh, you are in deep shit. You are in deep shit. And although there's rage, in my chest, I'm still passing the test. With a fuck, fuck, Jews. Who knew who? Yeah, they're greedy, and a little bit seedy. In fact they're the prime source of all degeneracy. I make two pizza, two pizza, and I distribute it quite generously. I pity the fool. That believes a word from Jews. Oh, these kikes are just gold diggers. Well, they want white women to fuck niggers. Ooh, hoo hoo. I've got some gas for you. Yego run and tell your little anti-deformation leak. And I see them feeding fake news to the people I love. And I'm like, fuck, Jews. Who knew who? I guess the six fucking Gorylian wasn't enough. I'm like, fuck, Jews, and

/Users/cassie/anaconda3/lib/python3.11/site-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Transcribed text for non_hate_video_584.mp3:  You hate yourself. You hate Israel but you're joke. You got to sit and have yourself. You're fucking... Yeah, man, when I put it that way, you're doing the power spinning. That shit ain't right. Yeah, that's right. Yes! That shit ain't right. You got things for you people. You got you, you got a pass. That's fucked out of you. You got a fucking... What? ZKM of a fucker. Oh...
Transcribed text saved to ../Datas/text_output/non_hate_video_584_text.txt
Converted audio saved to ../Datas/extract_audio_files/non_hate_video_58.wav


/Users/cassie/anaconda3/lib/python3.11/site-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Transcribed text for non_hate_video_58.mp3:  Now, how do I know that the white people know that we are going to come up with a solution to the problem? I know it because they have retina scans, they have what they call a ratio profile, DNA banks, and they're monitoring our people to try to prevent the one person from coming up with the one idea. The one idea is how we are going to exterminate white people because that in my estimation is the only conclusion I have come to. We have to exterminate white people off of the face of the planet to solve this problem. Now, I don't care whether you clap or not, but I'm saying to you that we need to solve this problem because they are going to kill us and I will leave on that. So we have to just set up our own system and stop playing and get very serious and not be diverted from coming up with a solution to the problem and the problem on the planet is white people.
Transcribed text saved to ../Datas/text_output/non_hate_video_58_text.txt
